# Hurriyet Üzerinden Haber İçeriklerinin Scrape Edilmesi

1. Burada öncelikle, hürriyetin günlük olarak haberlerini listelediği bir sayfa üzerinden 20 adet haberin url'ini alıp, ardından bu haberlerin içerisindeki verileri kazıyacağız.

In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as bs
import requests 
import datetime

In [68]:
def GetText(url):

    # Öncelikle haber linkine giriyoruz, Hurriyette birkaç farklı yapı var. Bunları tek tek deniyoruz. 
    html = requests.get("http://www.hurriyet.com.tr/"+url).text
    soup = bs(html, "lxml")
    try:
        #Bu kısımda ana metin olarak rhd-all-article-detail'ine bakıyoruz. 
        body = soup.find("div", class_="rhd-all-article-detail").findAll('p')
    except AttributeError:
        try:
            # Bu kısımda tek tek elemanlar var mı diye bakıyoruz, yoksa hata veriyor ve diğer bir tipte olduğunu anlayıp o şekilde ilerliyoruz.
            body_text = soup.findAll("div", class_="news-box")[1].find('p').text
            summarized_text=soup.findAll("div", class_="news-detail-spot news-detail-spot-margin")[0].find('h2').text
            header = soup.find("h2", class_="news-detail-title selectionShareable local-news-title").text
            time = soup.find("div", class_="col-md-8 text-right").text[:10]
            return (body_text,summarized_text,header,time)
        except:
            header = soup.find("h1", class_="rhd-article-title").text
            time = soup.find("div", class_="rhd-time-box").text[:10]
            body = soup.findAll("h3", class_="description")
            body_text = ''
            for element in body:
                body_text += ''.join(element.findAll(text = True))
            summarized_text = soup.find("h2", class_="rhd-article-spot").text
            return (body_text,summarized_text,header,time)
    body_text = ''
    for element in body:
            body_text += ''.join(element.findAll(text = True))
    # Koruma
    if len(body_text) == 0:
        body_text = soup.find("div", class_="rhd-all-article-detail").text
        summarized_text = soup.find("h2", class_="rhd-article-spot").text
    try:
        header = soup.find("h2", class_="news-detail-title selectionShareable local-news-title").text
        time = soup.find("div", class_="col-md-8 text-right").text[:10]
    except AttributeError:
        header = soup.find("h1", class_="rhd-article-title").text
        time = soup.find("div", class_="rhd-time-box").text[:10]
        summarized_text = soup.find("h2", class_="rhd-article-spot").text
        return (body_text,summarized_text,header,time)
    return (body_text,summarized_text,header,time)

In [69]:
# Bu kısımda o güne ait 40 içeriği alacağız
url = 'http://www.hurriyet.com.tr/index/?d=20191130'
html = requests.get(url).text
soup = bs(html, "lxml")

# Bu kısımda news tag'ının altındaki bütün linkleri topluyoruz. Bunlarda haber linklerimiz oluyor.
urlx = []
for links in soup.findAll("div", class_="news"):
    urlx.append(links.find('a').get('href'))

# 10 Adet url'i alacağız.
url = urlx[10:20]
    
# Bazı urller sadece video oluyor, bunlarda metin içeriği olmadığı için işlemek çok zor oluyor.
for c in url:
    if c[:6] == "/video":
        url.remove(c)
        url.append(urlx[np.random.randint(20,len(urlx))])
    else:
        continue

# Bu kısımda url'lerin verilerini topluyoruz.
data = []
for i in url:
    data.append(GetText(i))

# veriyi pandas df haline getiriyoruz.
data = pd.DataFrame(data)
data.columns = ["body_text","summarized_text","header","time"]
data.head()

,body_text,summarized_text,header,time
0,"Bayer Münih, Bundesliga’nın 13. haftasında All...",Almanya Birinci Futbol Ligi’nde (Bundesliga) B...,Bayern Münih'e Leverkusen darbesi!,30.11.2019
1,FOX TV ekranlarında pazar akşamları diziseverl...,"Berk Oktay, Murat Serezli, Sarp Levendoğlu, Mü...",Savaşçı'nın 84. bölümünde neler olacak? İşte y...,30.11.2019
2,Romanya’nın Başkenti Bükreş’te çekilen EURO 20...,EURO 2020’de Türkiye ile aynı grupta mücadele ...,Vladimir Petkoviç: Türkiye çok iyi bir jeneras...,30.11.2019
3,SIM kartlar yolun sonuna geliyor; artık telefo...,"SIM kartlar tarih oluyor, işte milyonlarca tel...",Telefon sahiplerine SIM kart uyarısı! Değişiyo...,30 Kasım 2
4,Instagram'daki crazyinrussia hesabı da Rusya'd...,Rusya'da çekilen ilginç görüntüler her zaman s...,"""Putin'in gençliği"" metroya bindi, ortalık kar...",30 Kasım 2


In [70]:
data

,body_text,summarized_text,header,time
0,"Bayer Münih, Bundesliga’nın 13. haftasında All...",Almanya Birinci Futbol Ligi’nde (Bundesliga) B...,Bayern Münih'e Leverkusen darbesi!,30.11.2019
1,FOX TV ekranlarında pazar akşamları diziseverl...,"Berk Oktay, Murat Serezli, Sarp Levendoğlu, Mü...",Savaşçı'nın 84. bölümünde neler olacak? İşte y...,30.11.2019
2,Romanya’nın Başkenti Bükreş’te çekilen EURO 20...,EURO 2020’de Türkiye ile aynı grupta mücadele ...,Vladimir Petkoviç: Türkiye çok iyi bir jeneras...,30.11.2019
3,SIM kartlar yolun sonuna geliyor; artık telefo...,"SIM kartlar tarih oluyor, işte milyonlarca tel...",Telefon sahiplerine SIM kart uyarısı! Değişiyo...,30 Kasım 2
4,Instagram'daki crazyinrussia hesabı da Rusya'd...,Rusya'da çekilen ilginç görüntüler her zaman s...,"""Putin'in gençliği"" metroya bindi, ortalık kar...",30 Kasım 2
5,Özellikle listede eksi 30 ile 45 derece soğukl...,"Güneşin teğet geçtiği, kar yağışının durmadığı...",Eksi 45 derece sıra dışı hayatlar! Sadece 600 ...,30 Kasım 2
6,Romanya’nın Başkenti Bükreş’te yapılan EURO 20...,Türkiye Futbol Federasyonu Başkanı Nihat Özdem...,"Nihat Özdemir: ""Bu grubu çok istemiştim""",30.11.2019
7,Star TV'de yayın hayatına başlayan ve pazar ak...,"Kadrosunda Mehmet Ali Nuroğlu, Almila Ada, Men...",Güvercin'in 2. bölümünde neler olacak? Yeni bö...,30.11.2019
8,Geçen hafta oynanan Fenerbahçe - Yeni Malatyas...,"Fenerbahçe Spor Kulübü, Twitter hesabından yap...",Son dakika: Fenerbahçe'den Göztepe maçıyla ilg...,30.11.2019
9,"İlk kez 12 ülkenin ev sahipliği yapacağı, 12 H...","A Milli Futbol Takımı, 2020 Avrupa Futbol Şamp...",İtalya Türkiye EURO 2020 açılış maçı ne zaman?,30.11.2019
